In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import gzip
import json

import pandas as pd

from matpowercaseframes import CaseFrames

## UnitCommitment.jl Data

In [ ]:
CASE_NAME = "../data/case14/2017-01-01.json.gz"
with gzip.open(CASE_NAME, "rt", encoding="utf-8") as f:
    data = json.load(f)

In [4]:
# TODO: convert `UnitCommitment.jl` format to matpowercaseframes format
data.keys()

dict_keys(['SOURCE', 'Parameters', 'Generators', 'Transmission lines', 'Contingencies', 'Buses', 'Reserves'])

In [5]:
data["Generators"]["g1"]

{'Bus': 'b1',
 'Production cost curve (MW)': [36.75123365792352,
  110.10634590601819,
  183.46145815411288,
  256.8165704022076,
  330.1716826503023],
 'Production cost curve ($)': [1160.1370560288465,
  4051.592909653724,
  7310.413973593571,
  10614.711162105745,
  14293.941140664052],
 'Startup costs ($)': [25056.51, 29059.15, 34238.45],
 'Startup delays (h)': [1, 2, 4],
 'Ramp up limit (MW)': 231.12,
 'Ramp down limit (MW)': 231.12,
 'Startup limit (MW)': 231.12,
 'Shutdown limit (MW)': 231.12,
 'Minimum uptime (h)': 1,
 'Minimum downtime (h)': 1,
 'Reserve eligibility': ['r1'],
 'Initial status (h)': 24,
 'Initial power (MW)': 230.74887999999999}

## MATPOWER Data

In [6]:
CASE_NAME = "case14.m"
cf_14 = CaseFrames(CASE_NAME)

In [7]:
cf_14.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1.0,232.4,-16.9,10.0,0.0,1.060,100.0,1.0,332.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,40.0,42.4,50.0,-40.0,1.045,100.0,1.0,140.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,23.4,40.0,0.0,1.010,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.0,0.0,12.2,24.0,-6.0,1.070,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,8.0,0.0,17.4,24.0,-6.0,1.090,100.0,1.0,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# TODO: apply to matpowercaseframes
def unitcommitmentjl_to_matpower(data: dict):
    gen_list = []
    for _gen_name, gen_data in data["Generators"].items():
        gen_bus = gen_data["Bus"]
        mw_points = gen_data["Production cost curve (MW)"]
        pmin = min(mw_points)
        pmax = max(mw_points)

        pg = gen_data.get("Initial power (MW)", 0.0)
        gen_status = 1 if gen_data.get("Initial status (h)", 0) > 0 else 0
        ramp_up = gen_data.get("Ramp up limit (MW)", 0.0)

        matpower_gen = {
            # MATPOWER fields
            "GEN_BUS": gen_bus,
            "PG": pg,
            "QG": 0.0,  # Not provided in UC data
            "QMAX": 0.0,  # Not provided in UC data
            "QMIN": 0.0,  # Not provided in UC data
            "VG": 1.0,  # Default value
            "MBASE": 100.0,  # Default MVA base
            "GEN_STATUS": gen_status,
            "PMAX": pmax,
            "PMIN": pmin,
            "PC1": 0.0,  # Not used
            "PC2": 0.0,  # Not used
            "QC1MIN": 0.0,  # Not used
            "QC1MAX": 0.0,  # Not used
            "QC2MIN": 0.0,  # Not used
            "QC2MAX": 0.0,  # Not used
            "RAMP_AGC": ramp_up / 60.0,
            "RAMP_10": ramp_up / 6.0,
            "RAMP_30": ramp_up / 2.0,
            "RAMP_Q": 0.0,  # Not provided
            "APF": 0.0,  # Not used
            # UnitCommitment.jl fields
            "RAMP_UP": gen_data.get("Ramp up limit (MW)", 0.0),
            "RAMP_DOWN": gen_data.get("Ramp down limit (MW)", 0.0),
            "RAMP_UP_SU": gen_data.get("Startup limit (MW)", 0.0),
            "RAMP_DOWN_SD": gen_data.get("Shutdown limit (MW)", 0.0),
            "MUT": gen_data.get("Minimum uptime (h)", 0),
            "MDT": gen_data.get("Minimum downtime (h)", 0),
            "GEN_STATUS_INIT": gen_data.get("Initial status (h)", 0),
            "PG_INIT": gen_data.get("Initial power (MW)", 0.0),
            "RESERVE": gen_data.get("Reserve eligibility", []),
        }
        gen_list.append(matpower_gen)

    return {
        "gen": pd.DataFrame(gen_list),
    }

In [ ]:
gen_list = unitcommitmentjl_to_matpower(data)
gen_df = pd.DataFrame(gen_list)
gen_df

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,APF,RAMP_UP,RAMP_DOWN,RAMP_UP_SU,RAMP_DOWN_SD,MUT,MDT,GEN_STATUS_INIT,PG_INIT,RESERVE
0,b1,230.74888,0.0,0.0,0.0,1.0,100.0,1,330.171683,36.751234,...,0.0,231.12,231.12,231.12,231.12,1,1,24,230.74888,[r1]
1,b2,0.00000,0.0,0.0,0.0,1.0,100.0,0,132.319116,2.257204,...,0.0,92.62,92.62,92.62,92.62,1,1,-24,0.00000,[r1]
2,b3,0.00000,0.0,0.0,0.0,1.0,100.0,0,94.583138,3.074136,...,0.0,66.21,66.21,66.21,66.21,1,1,-24,0.00000,[r1]
3,b6,0.00000,0.0,0.0,0.0,1.0,100.0,0,95.634291,2.982752,...,0.0,66.94,66.94,66.94,66.94,4,4,-24,0.00000,[r1]
4,b8,0.00000,0.0,0.0,0.0,1.0,100.0,0,92.911604,6.042765,...,0.0,65.04,65.04,65.04,65.04,1,1,-24,0.00000,[r1]
